# Práctica: clasificación automática de spam

<img src="img/Spam_can.png" style="width:400px;height:400px;">

En esta práctica vamos a construir un clasificador automático de spam, que dado un texto extraído de un mensaje SMS nos pueda decir si se trata de un correo legítimo o de un mensaje no deseado o fraudulento. Para ello vamos a utilizar métodos sencillos de tratamiento de textos, que sin embargo resultan ser muy efectivos para este problema.

## Instrucciones

A lo largo de este cuaderno encontrarás celdas vacías que tendrás que rellenar con tu propio código. Sigue las instrucciones del cuaderno y presta especial atención a los siguientes iconos:

<table>
<tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">Deberás responder a la pregunta indicada con el código o contestación que escribas en la celda inferior.</td></tr>
 <tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">Esto es una pista u observación que te puede ayudar a resolver la práctica.</td></tr>
 <tr><td width="80"><img src="img/pro.png" style="width:auto;height:auto"></td><td style="text-align:left">Este es un ejercicio avanzado y voluntario que puedes realizar si quieres profundar más sobre el tema. Te animamos a intentarlo para aprender más ¡Ánimo!</td></tr>
</table>

Para evitar problemas de compatibilidad y de paquetes no instalados, se recomienda ejecutar este notebook bajo uno de los [entornos recomendados de Text Mining](https://github.com/albarji/teaching-environments/tree/master/textmining).

Adicionalmente si necesitas consultar la ayuda de cualquier función python puedes colocar el cursor de escritura sobre el nombre de la misma y pulsar Mayúsculas+Shift para que aparezca un recuadro con sus detalles. Ten en cuenta que esto únicamente funciona en las celdas de código.

¡Adelante!

## Preliminares

En primer lugar vamos a fijar la semilla aleatoria para que los resultados sean reproducibles entre diferentes ejecuciones del notebook.

In [ ]:
import numpy as np
np.random.seed(12345)

## Carga y preparación de datos

Para construir nuestro clasificador automático de spam vamos a utilizar los datos que puedes encontrar en el fichero *SMSSpamCollection* de la carpeta *data*. En esa misma carpeta encontrarás un fichero *readme* con información sobre el origen de este conjunto de datos. Vamos a tener que cargar estos ficheros en Python, para lo que antes tendremos que entender cómo se estructura este fichero.

<table>
 <tr>
  <tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">
      Abre el fichero <i>SMSSpamCollection</i> con cualquier programa de edición de texto, y trata de descubrir:
     <ul>
      <li>¿Qué columnas de información tiene este fichero?</li>
      <li>¿Cuál es el caracter separador de columnas?</li>
      <li>¿Cómo están etiquetadas las clases de mensajes de spam y de mensajes legítimos?</li>
  </td>
 </tr> 
</table>

La forma más efectiva de cargar estos datos es usando la librería **pandas**, que no solo da funciones de lectura de ficheros sino también de manipulación de estos datos en memoria.

In [ ]:
import pandas as pd

La función más adecuada para realizar la carga de este tipo de fichero es [read_csv](http://pandas.pydata.org/pandas-docs/version/0.16.2/generated/pandas.read_csv.html).

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
      Usando la función <b>read_csv</b>, carga en memoria los datos en una variable con nombre <b>data</b>.
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

Vamos ahora a comprobar si se ha realizado la carga de los datos correctamente. La función **read_csv** devuelve un objeto de tipo DataFrame, que permite analizar los datos con facilidad. El siguiente código debería mostrar correctamente tanto los textos como las etiquetas de los 10 primeros mensajes:

In [ ]:
data.head()

<table>
 <tr>
  <tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">
    Comprueba que al ejecutar la celda anterior se muestran correctamente los datos.
  </td>
 </tr> 
</table>

A continuación vamos a separar estos datos en dos subconjuntos: un conjunto **train** para realizar el entrenamiento de nuestro modelo de clasificación de spam, y otro conjunto **test** sobre el que probar nuestro modelo para medir así el nivel de acierto de nuestra solución. Para esto vamos a tener que generar dos nuevos DataFrame, **train** y **test** que contengan el 75% y el 25% de los datos, respectivamente. Esto podemos hacerlo fácilmente usando la función apropiada de scikit-learn

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.25)

## Modelo sencillo basado en caracteres

### Convirtiendo el texto a vectores

Para empezar vamos a construir un modelo que simplemente tenga en cuenta el tipo de caracteres que aparecen en el texto para tratar de determinar si se trata de un mensaje de spam o legítimo. Para ello vamos a utilizar la estrategia de CountVectorizer que seguimos en la práctica anterior.

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
      Crea un objeto de tipo <b>CountVectorizer</b> que analize unigramas de caracteres, haciendo la cuenta de forma binaria. Después utiliza el vectorizador que has creado para convertir todos los textos del conjunto de entrenamiento <b>train</b>, guardando el resultado en la variable <b>X</b>, que será la matriz que usaremos para entrenar el clasificador automático.
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

<table>
 <tr>
  <tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">
    Para asegurarte de que has realizado la transformación correctamente puedes mirar cuál es el vocabulario que ha contruido el vectorizador, cuál es el contenido del primer texto de <b>train</b>, y ver si cuadra con la representación vectorial generada en la primera fila de <b>X</b>.
  </td>
 </tr> 
</table>

### Entrenando el clasificador automático

Ahora que tenemos los datos en formato vectorial vamos a construir un modelo con ellos. Como modelo vamos a utilizar una **SVM lineal**, que es un modelo muy frecuentemente utilizado junto con representaciones tipo bag-of-words debido a sus rápidos tiempos de entrenamiento y su robustez al enfrentarse con un número muy elevado de variables explicativas. Este modelo está también disponible en el paquete scikit-learn:

In [ ]:
from sklearn.svm import LinearSVC

Todos los modelos de scikit-learn funcionan de la misma manera, operando a través de los siguientes métodos:
* **fit**: recibe una matriz X de datos de entrenamiento, y una matriz o vector Y con las etiquetas que deseamos estimar. X debe tener el mismo número de filas que Y.
* **predict**: recibe una matrix X de datos de test, y genera una matriz o vector con las etiquetas más probables para cada dato de test. Para invocar este método antes debe haberse realizado el fit.
* **score**: recibe una matrix X de datos de test, y una matriz o vector Y con las etiquetas que debería estimar el modelo. Como resultado devuelve un score o medida de la precisión del modelo, que se calcula comparando las predicciones de modelo contra las etiquetas esperadas que se han proporcionado. Para invocar este método antes debe haberse realizado el fit.

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
      Entrena un modelo LinearSVC con los datos de entrenamiento <b>X</b> que has preparado. Para ello necesitarás crear un objeto del tipo LinearSVC y llamar a su método train con X y las etiquetas de tu DataFrame <i>train</i>.
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

Para comprobar que el modelo se ha entrenado correctamente, podemos medir el score que tiene sobre los propios datos de entrenamiento. Esta no es una forma adecuada de conseguir una medida realista del error, pero nos sirve para saber que no tenemos fallos en el proceso de entrenamiento.

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
    Calcula el score del modelo con los propios datos de entrenamiento. ¿Obtienes un nivel de score (precisión del clasificador) razonable?
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

### Evaluando el clasificador automático

Vamos ahora a medir cómo de bien funciona el clasificador sobre el conjunto de test.

Para ello vamos a tener que transformar los textos del conjunto de test a vectores siguiendo **exactamente** el mismo proceso que para el entrenamiento. Para conseguir esto tendremos que usar el vocabulario de palabras que se calculó cuando pasamos por el vectorizador los datos de entrenamiento. Esto podemos hacerlo usando la función **transform** de nuestro vectorizador, que realiza la transformación usando el vocabulario obtenido en la última llamada a **fit_transform**.

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
    Utiliza el vectorizador que creaste durante el entrenamiento para convertir todos los textos del conjunto de test <b>test</b>, guardando el resultado en la variable <b>Xtest</b>.
  </td>
 </tr> 
</table>

<table>
 <tr>
  <tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">
      Es importante <b>no reentrenar</b> el vector usando fit_transform para los datos de test. Si lo hacemos estaremos construyendo un vectorizador diferente, que será incompatible con el modelo que hemos preparado sobre los datos de entrenamiento.
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

Ahora que tenemos los datos de test en formato vectorial vamos a medir el acierto de nuestro modelo.

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
      Utiliza la función <b>score</b> del modelo que has creado para medir el acierto sobre los datos de test.
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

### Unificando el proceso

Existe una forma más eficaz de realizar todo el proceso de transformar datos de entrenamiento, construir el modelo, transformar los datos de test y medir la precisión de nuestro modelo sobre ellos. Esa forma es usar un **Pipeline** de scikit-learn:

In [ ]:
from sklearn.pipeline import Pipeline

Un Pipeline nos permite encadenar varios procesos de modelado, de forma que se ejecuten de forma coordinada. Por ejemplo, para encadenar un vectorizador con una SVM no tenemos más que crear el Pipeline de la siguiente manera:

In [ ]:
pipelineejemplo = Pipeline([
    ('vectorizer', CountVectorizer(analyzer = "word", ngram_range = (1,1))),
    ('classifier', LinearSVC())
    ]
)

El Pipeline se construye suministrando una lista de tuplas, cada tupla conteniendo un elemento procesador y un nombre que decidamos asignarle. Una vez construído podemos ejecutar las tareas de entrenamiento, que incluyen la transformación de los textos a vectores, ejecutando la función **fit** del pipeline, para después hacer predicciones o medir la precisión del modelo sobre datos de test con las funciones **predict** y **score**. En definitiva, un Pipeline se emplea prácticamente de la misma manera que un modelo simple de scikit-learn.

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
      Construye un <b>Pipeline</b> que incluya un vectorizador de unigramas caracteres como el que has usado antes, y una SVM lineal como modelo de clasificación. A continuación entrena el pipeline usando los datos de entrenamiento y calcula el score para los datos de test. Ten en cuenta que como el Pipeline ya incluye el paso de transformación de textos a vectores, los datos que debes suministrar para el entrenamiento y el scoring no son las matrices X y Xtest, sino la lista de textos originales.
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

<table>
 <tr>
  <tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">
    Si lo has hecho todo correctamente deberías haber obtenido el mismo score que en el apartado anterior.
  </td>
 </tr> 
</table>

¿Cuál es la utilidad de agrupar todo en un Pipeline como hemos hecho? Principalmente la facilidad de uso que nos da para poder probar varias opciones de procesado y modelado rápidamente. En los siguientes apartados vamos a explotar esto.

## Modelos de n-gramas más avanzados

Ahora que hemos construído un modelo sencillo y tenemos una estimación de lo bien que funciona vamos a probar estrategias más avanzadas para comprobar si podemos mejorar nuestro nivel de precisión a la hora de detectar spam. Para ello vamos a experimentar cambiando las opciones de vectorización de nuestro pipeline, buscando qué estrategias funcionan mejor para el problema.

Lo primero que vamos a hacer es cambiar el modelo de uni-gramas por uno de bi-gramas.

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
      Construye un nuevo <b>Pipeline</b> similar al del apartado anterior pero que en lugar de utilizar solo unigramas incluya tanto <b>unigramas como bigramas</b>. ¿Qué score obtienes sobre los datos de test? ¿Es mejor que el alcanzado por el modelo que solo emplea unigramas?
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
      Construye ahora otro <b>Pipeline</b> que en lugar de construir n-gramas de caracteres lo haga sobre palabras ¿Qué score obtienes sobre los datos de test? ¿Es mejor que el anterior?
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
      Ahora intenta construir el mejor <b>Pipeline</b> de procesado posible. Para ello juega con todas las opciones que existen en CountVectorizer. ¿Cuál es el mejor nivel de score que puedes obtener?
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

## Automatizando la selección de parámetros

Una de las ventajas de scikit-learn es que podemos implementar fácilmente un proceso que automáticamente busque qué parametros son los mejores para nuestro modelo. Esto incluye tanto las opciones que dirigen nuestro vectorizador como los parámetros del modelo de clasificación que estamos utilizando. Una de las formas de hacer esto es utilizando un proceso de validación cruzada, como:

In [ ]:
from sklearn.model_selection import GridSearchCV

El objeto **GridSearchCV** realiza automáticamente un proceso de validación cruzada de k-hojas sobre el modelo o pipeline que proporcionemos, probando toda las posibles combinaciones de parámetros y estimando su precisión. Una vez que la combinación de modelos de parámetros con mayor precisión se ha encontrado, el modelo o pipeline se entrena con esos parámetros usando el dataset completo.

Cuando trabajamos con un pipeline podemos indicar los parámetros de los componentes del pipeline a optimizar como el nombre del componente seguido por dos guiones bajos y el nombre del parámetro. Por ejemplo, para hacer un GridSearchCV que optimice el parámetro C de la SVM lineal y el parámetro binary del vectorizador tendríamos que escribir:

In [ ]:
# Declaración del Pipeline
pipelineejemplo = Pipeline([
    ('vectorizer', CountVectorizer(analyzer = "word", ngram_range = (1,1))),
    ('classifier', LinearSVC())
    ]
)

# Declaración de los parámetros a optimizar automáticamente, y los valores que se probarán para cada uno
paramsejemplo = {
    'classifier__C': [0.1, 1, 10],
    'vectorizer__binary' : [False, True],
}

# Declaración de la estrategia de validación cruzada
modelejemplo = GridSearchCV(pipelineejemplo, paramsejemplo)

# Ejecución de todo el proceso de entrenamiento, incluída la búsqueda de parámetros
modelejemplo.fit(train["text"].values, train["class"])

Y con esto hemos obtenido un modelo que automáticamente ha seleccionado los mejores valores para los parámetros que hemos indicado. Podemos averiguar qué parametros son estos mediante inspección del objeto de modelo que hemos entrenado:

In [ ]:
modelejemplo.best_params_

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
    Imitando el ejemplo anterior construye un pipeline para el que mediante validación cruzada se optimice el parámetro C de la SVM, y los parámetros binary y analyzer del vectorizador. Para ello consulta más arriba cuáles son los valores posibles para el parámetro analyzer. Ten en cuenta que al declarar el vectorizador o el clasificador no es necesario asignar los parámetros que luego vayan a optimizarse en la validación cruzada. Tras el entrenamiento calcula el score sobre el conjunto de test. ¿Has conseguido superar tu mejor resultado?
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

<table>
 <tr>
  <tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">
    ¿El proceso de entrenamiento tarda mucho? Al crear el objeto GridSearchCV puedes pasarle un parámetro <b>n_jobs</b> en el que indicarle el número de procesos que usar para el cálculo. Puedes incrementarlo para acelerar el cálculo, a cambio de usar más procesadores de tu máquina. ¡Procura no exceder tu número de procesadores!
  </td>
 </tr> 
</table>

## Otras formas de vectorización

Es posible utilizar alternativas al método CountVectorizer que hemos estado empleando hasta ahora. Mientras que CountVectorizer esencialmente implementa una estrategia de "bag-of-words" para generar un vector que represente el texto, es posible también utilizar estrategias tipo "tf-idf" o basadas en el "hashing trick".

### tf-idf

Term Frequency - Inverse Document Frequency (tf-idf) es una técnica similar a bag-of-words, pero en la que se **pesan** cada una de las palabras o tokens según la relevancia que parezcan tener en el texto siendo analizado. Esta relevancia se calcula según el número de veces que la palabra aparece en el documento (Term Frequency) dividido por el número de documentos del corpus en los que aparece ese mismo término (Inverse Document Frequency). Esto nos permite reducir la importancia que tendrán en nuestro modelo las palabras generales del idioma que aparecen en casi todos los documentos (stop-words), centrándonos en palabras más distintivas de cada texto.

En scikit-learn el vectorizador de textos mediante tf-idf es la clase [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

Podemos emplearla de forma similar a CountVectorizer, como se muestra en este ejemplo

In [ ]:
vectorizadorejemplo = TfidfVectorizer()
ejemplos = [
    "The cat sat on the mat",
    "The dog barked at the cat",
    "Dog days"
]
transformados = vectorizadorejemplo.fit_transform(ejemplos)
transformados.toarray()

En la vectorización resultado se observa cómo cada palabra cuenta con pesos diferentes.

Podemos además inspeccionar el vectorizador para ver qué vocabulario se ha construído:

In [ ]:
vectorizadorejemplo.vocabulary_

Igualmente es posible ver qué valores de Inverse Document Frequency (idf) se han calculado para cada palabra de este vocabulario. El siguiente código recupera del vectorizador los nombres de cada una de las características generadas (get_feature_names_out) y sus pesos idf:

In [ ]:
list(zip(vectorizadorejemplo.get_feature_names_out(), vectorizadorejemplo.idf_))

Como podemos ver, palabras como "days" o "mat" tienen mayor puntuación al aparcer en pocos documentos del corpus.

Para entrenar un modelo usando este vectorizador podemos usar la misma estrategia de Pipeline que ya hemos visto:

In [ ]:
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(ngram_range=(1,1))),
    ('classifier', LinearSVC())
    ]
)
pipeline.fit(train["text"].values, train["class"])
pipeline.score(test["text"].values, test["class"])

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
    Siguiendo el proceso realizado para CountVectorizer, construye un pipeline para el que mediante validación cruzada se optimice el parámetro C de la SVM, y los parámetros binary y ngram_range del TfidfVectorizer. Para el parámetro ngram_range usa los valores: (1,1), (1,2) y (1,3). ¿Son los resultados mejores que con CountVectorizer?
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

Una vez entrenado el modelo es posible extraer el mejor vectorizador encontrado sacándolo del Pipeline por su nombre:

In [ ]:
model.best_estimator_.named_steps["vectorizer"]

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
    Analiza el vectorizador obtenido en la optimización anterior y extrae la lista de las 20 palabras con menor peso idf. ¿Las palabras con menor peso idf parecen palabras generales del lenguaje?
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

### Hashing trick

Otra técnica alternativa a las estrategias basadas en diccionarios como CountVectorizer o tf-idf es usar una función de hash que para cada palabra a token indique un índice en un vector de características. Esto puede hacerse mediante la clase [HashingVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html)

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer

In [ ]:
vectorizadorejemplo = HashingVectorizer(n_features=5)
ejemplos = [
    "The cat sat on the mat",
    "The dog barked at the cat",
    "Dog days"
]
transformados = vectorizadorejemplo.fit_transform(ejemplos)
transformados.toarray()

Scikit-learn emplea una función de hash que puede devolver tanto valores negativos como positivos, además de aplicar una normalización del vector resultado, lo que explica que se obtengan números no enteros. Además en este ejemplo hemos empleando un tamaño de vector de características muy pequeño, ya que lo habitual en esta técnica es generar cientos de miles o millones de características.

Una vez más, podemos usar un Pipeline para entrenar un modelo que emplee este vectorizador

In [ ]:
pipeline = Pipeline([
    ('vectorizer', HashingVectorizer()),
    ('classifier', LinearSVC())
    ]
)
pipeline.fit(train["text"].values, train["class"])
pipeline.score(test["text"].values, test["class"])

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
    Siguiendo el proceso realizado para CountVectorizer, construye un pipeline para el que mediante validación cruzada se optimice el parámetro C de la SVM, y los parámetros binary y ngram_range del HashingVectorizer. Para el parámetro ngram_range usa los valores: (1,1), (1,2) y (1,3). ¿Son los resultados mejores que con CountVectorizer?
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

### Bonus round: no rest for the spammers

<table>
 <tr>
  <tr><td width="80"><img src="img/pro.png" style="width:auto;height:auto"></td><td style="text-align:left">
    Explora todos los parámetros de los que disponen CountVectorizer, TfidfVectorizer y HashingVectorizer, y trata de encontrar el mejor modelo posible. ¡Ten cuidado porque el coste de optimización crece exponencialmente con el número de parámetros! ¿Cuál es el mejor score que puedes conseguir?
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE